# Curso BancoEstado-CMM: Aprendizaje de Máquinas Avanzado 
**Autores:** Taco de Wolff, Nicolas Caro y Felipe Tobar  

**Fecha:** 27 deciembre, 2019

# Módulo 9 - Redes Bayesianes
Instalar con `conda install pomegranate`.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set_style('whitegrid')
import numpy as np

from pomegranate import *

## Introducción

El eje central de este modulo se desarrolla en torno a los modelos gráficos
probabilísticos (PGM). En términos generales, este tipo de modelo busca una
representación compacta para distribuciones de probabilidad conjunta de la
forma $p(\mathbf{x} | \mathbf{\theta})$, valiéndose de los fundamentos matemáticos
proporcionados por la teoría de grafos y de probabilidad.

Los PGM son ampliamente utilizados en tareas de razonamiento bajo incertidumbre,
tales como predicción, monitorio y diagnósticos médicos, además de tareas de
estimación de riesgos y toma de decisiones.


### Modelos gráficos dirigidos

Toda distribución de probabilidad conjunta $p(\mathbf{x} ) = p(x_1, x_2, \ldots x_v)$  se puede
representar de la forma:
\begin{equation}
	\label{eq:regla_cadena}
	p(\mathbf{x} ) = p(x_1)p(x_2 | x_1 )p(x_3 |  x_1, x_2) \ldots p(x_v |  x_1, x_2, \ldots , x_{v-1})
\end{equation}

El problema con esta expresión es la dificultad computacional subyacente al cálculo de
distribuciones condicionales de la forma $p(x_{t} |  x_1,\ldots, x_{t-1} )$
cuando el número de variables incidentes $t$ aumenta.

En efecto, si se asume $x_{t+1} \perp  x_{1}, \ldots, x_{t-1} | x_t$. Es decir,
las observaciones futuras $x_{t+1}$ son independientes del pasado $ x_{1}, \ldots,
x_{t-1}$, dado el estado presente $x_{t}$. La probabilidad conjunta se reduce entonces a:

\begin{equation}
\label{eq:markov_prop}
p(\mathbf{x} ) = p(x_1) \prod_{t = 2}^{v} p(x_t  |  x_1,\ldots , x_{t-1}) = p(x_1) \prod_{t = 2}^{v} p(x_t  | x_{t-1})
\end{equation}

De lo cual se obtiene una expresión más simple.

Modelar la independencia condicional entre las variables permite entonces reducir la
complejidad de representación para la distribución conjunta. En particular, la elección anterior se conoce como **propiedad de
Markov**.

En un contexto general, las relaciones de independencia condicional entre variables aleatorias
de dimensión arbitraria, se modelan utilizando **diagramas de independencia** o **modelos gráficos**. Estos se valen de un conjunto
consistente de $\mathcal{V} = \lbrace{1 \ldots , V} \rbrace$ vértices (o nodos) y $\mathcal{E} =\lbrace {(s,t): s,t \in \mathcal{V}} \rbrace$ aristas  para formar una estructura matemática conocida como **Grafo** $G=(\mathcal{V}, \mathcal{E})$.

Los grafos permiten representar mediante nodos $v = 1, \ldots, \mathcal{V}$ las variables aleatorias del modelo,
mientras que la presencia o ausencia de aristas entre estos nodos, permite teorizar sobre las relaciones de dependencia condicional entre variabes de un modelo.

Una **red bayesiana** es un modelo gráfico probabilístico, cuyo grafo subyacente es un **grafo dirigido acíclico** (DAG). Todo DAG posee un **ordenamiento topológico**, es decir, los nodos de cualquier DAG pueden ser numerados de manera tal, que todo nodo padre posea una numeración inferior a sus nodos hijos. Esta característica permite enriquecer la formulación de la propiedad de Markov, usando la estructura gráfica como componente adicional, de
esta forma, se puede reformular a una **propiedad ordenada de Markov** en modelos gráficos dirigidos:

\begin{equation}
	\label{eq:Markov_ordenado}
	x_s \perp  x_{pred(s) \setminus pa(s)} ~ | ~  x_{pa(s)}
\end{equation}

Es decir, un nodo $x_s$ es independiente de aquellos predecesores, menores en orden topológico,
a sus padres $ x_{pred(s) \setminus pa(s)}$, dados sus nodos padres $x_{pa(s)}$. De
manera equivalente, un nodo $x_s$ solo depende de sus padres inmediatos $x_{pa(s)}$ y no de
todos sus predecesores.

De esta forma, la probabilidad conjunta de un modelo gráfico dirigido, que cumple la **propiedad
ordenada de Markov**, se puede descomponer de la forma:
\begin{equation}
\label{eq:markov_prop_ord}
p(\mathbf{x} ) = \prod_{t = 1}^{V} p(x_t  | x_{pa(t)})
\end{equation}

Con el fin de explorar las posibilidades de este tipo de modelos e introducir conceptos
referentes a la notación de estos, se pasan a estudiar los siguientes ejemplos:


## Monty Hall

Monty Hall fue un show de juegos en estados unidos, en el que se daba al concursante a elegir entre tres puertas, una de ellas permitía ganar auto como premio, las otras dos conducían a una cabra. Después elegir una puerta, el anfitrión abre una de las otras dos puertas en las que hay cabra, luego ofrece la oportunidad al concursante de cambiar su puerta previamente seleccionada. El problema consiste en encontrar una estrategia que maximice la probabilidad de ganar el auto.¿Conviene más cambiar de elección, mantener la elección o realmente no se ve afectada la probabilidad de ganar al cambiar de elección?

Es intuitivo creer que al cambiar de puerta no se afecta la probabilidad de ganar, pero en realidad la estrategia ganadora corresponde a **siempre cambiar de puerta**. Con redes bayesianas es posible explorar este problema.

El problema explicado en Wikipedia: https://en.wikipedia.org/wiki/Monty_Hall_problem

### Interpretación probabilistica

Supongamos que se tienen tres variables aleatorias: Concursante $ C \in \{1, 2, 3\} $, Anfitrión $ H \in \{1, 2, 3\} $ y el premio $ P \in \{1, 2, 3 \} $. La probabilidad de que el premio esté detrás de una puerta aleatoria viene dada por $ P(P=1) = P(P=2) = P(P=3) = \frac{1}{3} $. Además, el concursante seleccionará una puerta aleatoriamente, luego $ P(C=1) = P(C=2) = P(C=3) = \frac{1}{3} $. Construimos una red Bayesiana así:

In [ ]:
from IPython.display import Image
Image('monty.png')

In [ ]:
# Los concursantes eligen la puerta inicialmente aleatoriamente
guest = DiscreteDistribution({'A': 1./3, 'B': 1./3, 'C': 1./3})

# La puerta con el premio es aleatorio
prize = DiscreteDistribution({'A': 1./3, 'B': 1./3, 'C': 1./3})

# El anfitrión depende del concursante y el premio 
'''
La tabla de probabilidad condicional se construye con la siguiente 
estructura:

E1 : primera elección
P  : Posción del premio 
Eh : elección del host
Pr : Probabilidad de elegir Eh dado E1 y P

[E1, Prize, Eh, Pr]

Donde Pr es la probabilidad de elegir Eh dadas las elecciones E1 y P.

'''

host = ConditionalProbabilityTable(
        [[ 'A', 'A', 'A', 0.0 ],
         [ 'A', 'A', 'B', 0.5 ],
         [ 'A', 'A', 'C', 0.5 ],
         [ 'A', 'B', 'A', 0.0 ],
         [ 'A', 'B', 'B', 0.0 ],
         [ 'A', 'B', 'C', 1.0 ],
         [ 'A', 'C', 'A', 0.0 ],
         [ 'A', 'C', 'B', 1.0 ],
         [ 'A', 'C', 'C', 0.0 ],
         [ 'B', 'A', 'A', 0.0 ],
         [ 'B', 'A', 'B', 0.0 ],
         [ 'B', 'A', 'C', 1.0 ],
         [ 'B', 'B', 'A', 0.5 ],
         [ 'B', 'B', 'B', 0.0 ],
         [ 'B', 'B', 'C', 0.5 ],
         [ 'B', 'C', 'A', 1.0 ],
         [ 'B', 'C', 'B', 0.0 ],
         [ 'B', 'C', 'C', 0.0 ],
         [ 'C', 'A', 'A', 0.0 ],
         [ 'C', 'A', 'B', 1.0 ],
         [ 'C', 'A', 'C', 0.0 ],
         [ 'C', 'B', 'A', 1.0 ],
         [ 'C', 'B', 'B', 0.0 ],
         [ 'C', 'B', 'C', 0.0 ],
         [ 'C', 'C', 'A', 0.5 ],
         [ 'C', 'C', 'B', 0.5 ],
         [ 'C', 'C', 'C', 0.0 ]], [guest, prize])

# Los estados contienen la distribution y un nombre
s1 = State(guest, name="guest")
s2 = State(prize, name="prize")
s3 = State(host, name="host")

In [ ]:
# Creamos la red Bayesiana
model = BayesianNetwork("Monty Hall Problem")

# Agregamos los estados (nodos de grafo) 
model.add_states(s1, s2, s3)

# Agregamos las aristas entre los nodos. El segundo nodo depende del primero nodo.
# El anfitrión depende del concursante y el premio
model.add_edge(s1, s3)
model.add_edge(s2, s3)

# Construir el modelo
model.bake()

### Calcular probabilidades
Supongamos que el concursante elige puerta A, y el anfitrión muestra puerta C. ¿Qué es la probabilidad que el premio is detrás puerta A o B, y nos conviene cambiar a puerta B?

Sí, la probabilidad que el auto es detrás puerta B es $\frac{2}{3}$ mientras la probabilidad que el auto es detrás A queda con $\frac{1}{3}$.

In [ ]:
model.predict_proba({'guest': 'A', 'host': 'C'})

¿Qué pasa si solo el anfitrión muestra puerta A aunque el concursante no elegió nada? Hay igual probabilidad que el auto es detrás B o C.

In [ ]:
model.predict_proba({'host': 'A'})

### Naive Bayes

Dado un problema de clasificación de vectores $\mathbf{x} = (x_1, \ldots , x_V)$ en $C$ clases. Es
posible modelar las variables de decisión $x_t$ como condicionalmente independientes dada la
categoría de clasificación:

\begin{equation}
	\label{eq:cond_nb}
	x_i \perp x_j ~| y =  c, ~~ i \neq j
\end{equation}

Si se usa este enfoque, se obtiene que la densidad condicional de clases toma la forma:

\begin{equation}
	\label{eq:naive_bayes}
	p(\mathbf{x} ~ | y = c) = \prod_{t=1}^{V}p(x_t ~ | y = c)
\end{equation}

Al parametrizar las distribuciones de densidad condicional, es posible obtener un modelo de
clasificación conocido como **clasificador Naive Bayes**, por medio de este, es posible clasificar mediante:

\begin{equation}
	p(y = c | \mathbf{x} = (x_1,\ldots,x_V)) =\frac{p(c)}{p(\mathbf{x})}\cdot\prod_{t=1}^{V}p(x_t ~ | y = c)
\end{equation}

La estructura de las relaciones de independencia inducidas por tal método. se pueden expresar según el siguiente grafo.

In [ ]:
from IPython.display import Image
Image('naive_bayes.png')

¿Como se relaciona con el ejemplo de Monty Hall?

A continuación se entrena un clasificador Naive Bayes. En este caso se tiene la altura, el peso, y el tamaño de los pies para clasificar si cierta persona es hombre o mujer.

In [ ]:
male = NormalDistribution.from_samples([1.75, 1.72, 1.67, 1.81, 1.76, 1.71])
female = NormalDistribution.from_samples([1.62, 1.59, 1.54, 1.68, 1.60, 1.61])

In [ ]:
male.plot(n=100000, edgecolor='c', color='c', bins=50, label='Hombre')
female.plot(n=100000, edgecolor='g', color='g', bins=50, label='Mujer')
plt.legend(fontsize=14)
plt.ylabel('n')
plt.xlabel('Altura (m)')
plt.show()

print("Hombre: distribución tiene mu = %4.2f m y sigma = %5.3f m" % (male.parameters[0], male.parameters[1]))
print("Mujer: distribución tiene mu = %4.2f m y sigma = %5.3f m" % (female.parameters[0], female.parameters[1]))

`X` contiene la altura, el peso, y el tamaño de los pies para 12 personas. `y` dice `0` para un hombre y `1` para una mujer. Entrenamos nuestro modelo de Bayes ingenuo y distribuciones Gaussianes.

In [ ]:
X = [
    [1.75, 70, 11],
    [1.72, 82, 12],
    [1.67, 95, 12],
    [1.81, 105, 13],
    [1.76, 85, 10],
    [1.71, 72, 11],
    [1.62, 55, 9],
    [1.59, 64, 6],
    [1.54, 59, 8],
    [1.68, 68, 7],
    [1.60, 75, 9],
    [1.61, 62, 8],
]
y = [0,0,0,0,0,0,1,1,1,1,1,1]

clf = NaiveBayes.from_samples(NormalDistribution, X, y)

In [ ]:
data = np.array([[1.85, 80, 12], [1.70, 105, 11], [1.675, 69, 10], [1.65, 70, 7], [1.60, 65, 8]])

for sample, probability in zip(data, clf.predict_proba(data)):
    print("[%5.3f m, %3.0f kg, %2.0f ft]: %9.5f%% de probabilidad Hombre y %9.5f%% de probabilidad Mujer" % (sample[0], sample[1], sample[2], 100*probability[0], 100*probability[1]))

In [ ]:
for sample, result in zip(data, clf.predict(data)):
    print("Persona con [%5.3f m, %3.0f kg, %2.0f ft] es %s." % (sample[0], sample[1], sample[2], "Mujer" if result else "Hombre"))

## Estructura de Redes Bayesianas

La estructura de una red bayesiana contiene las relaciones de independencia condicional relativos a una probabilidad condicional. De esta forma, si $X,Y,Z$ son variables aleatorias es posbile poseer relaciones de dependecia de la forma:

* Secuencial:  $X \rightarrow Y \rightarrow Z$
* Divergente:  $X \leftarrow Y \rightarrow Z$
* Convetgente: $X \rightarrow  Y \leftarrow Z$

En los primeros 2 casos, $X$ y $Z$ son condicionalmente independientes dada $Y$. Sin embargo, en el tercer caso, se tiene el fenomeno conocido como *explaining away* el cual corresponde a tener 2 *causas* asociadas a una misma *consecuencia*, por lo tanto, al conocer el efecto de una de las causas y la consecuencia, se altera la certidumbre sobre la causa última. 

In [ ]:
# Ejemplo de red Bayesiana con estructura más Compleja

# Inferencia en Redes Bayesianas

# Aprendizaje en Redes Bayesianas